In [81]:
import multiprocessing as mp

In [82]:
mp.cpu_count()

4

In [160]:
# get a list of all imdb id's
import pandas as pd
import numpy as np
from imdb import IMDb

path = '/Users/austinbenny/Documents/python/movie budget ratings/IMDB_data/essential_imdb.csv'
df=pd.read_csv(path)
# replace nan and inf
df.replace([np.inf, -np.inf], np.nan)
df=df.dropna()
# reset the index
df.reset_index(drop=True, inplace=True)
# remove tt and extract into lists

In [161]:
# create list of global vars
global imdb_id
imdb_id=[df.loc[s,'imdb_id'].replace('tt','') for s in range(len(df))]

global title
title=df.loc[:,'title']

global inputs
inputs=['title','Cumulative Worldwide Gross','Budget','Opening Weekend United States','rating','runtimes','votes','aspect ratio']

global ia
ia=IMDb()

In [162]:
# the pool class should distribute them evenly
pool = mp.Pool(processes=mp.cpu_count())

In [167]:
# make funtion that takes only one input and that takes a list of ids and puts into df

def get_info(index):
    
    
    size=len(inputs)
    row=[np.nan]*size
    current_movie=ia.get_movie(imdb_id[index])
    
    if (ia.get_movie(imdb_id[index]).get('box office')==None) or (len(ia.get_movie(imdb_id[index]).get('box office'))<3) or (ia.get_movie(imdb_id[index])['box office'].get('Opening Weekend United States')==None):
        
        return row
        
    else:
         
        col=0
        for wants in inputs:
            
            if wants =='Budget':
                budget = current_movie['box office'][wants]
                budget=budget.split(' ', 1)[0]
                chars_remove='$,'
                for character in chars_remove:
                    budget = budget.replace(character, '')
                row[col]=float(budget)
            
            elif wants == 'Cumulative Worldwide Gross':
                gross = current_movie['box office'][wants]
                gross=gross.split(' ', 1)[0]
                chars_remove='$,'
                for character in chars_remove:
                    gross = gross.replace(character, '')
                row[col]=float(gross)
                
            elif wants == 'Opening Weekend United States':
                opening=current_movie['box office'][wants]
                opening=opening.split(' ', 1)[0]
                chars_remove='$,'
                for character in chars_remove:
                    opening = opening.replace(character, '')
                row[col]=float(opening)
            
            elif wants == 'runtimes':
                row[col]=current_movie[wants][0]
            
            elif wants == 'title':
                row[col]=title[index]
            
            else:
                row[col]=current_movie[wants]
                
            col+=1      
            
        return row
    
    

In [168]:
# test function

result=[get_info(s) for s in range(0,8)]
#result=get_info(0)
result

[['Toy Story',
  373554033.0,
  30000000.0,
  12491789.0,
  8.3,
  '81',
  841347,
  '1.78 : 1'],
 ['Jumanji',
  262797249.0,
  50000000.0,
  11084370.0,
  7.0,
  '104',
  289088,
  '1.85 : 1'],
 [nan, nan, nan, nan, nan, nan, nan, nan],
 [nan, nan, nan, nan, nan, nan, nan, nan],
 [nan, nan, nan, nan, nan, nan, nan, nan],
 ['Heat', 120000000.0, 60000000.0, 8445656.0, 8.2, '170', 546627, '2.35 : 1'],
 [nan, nan, nan, nan, nan, nan, nan, nan],
 [nan, nan, nan, nan, nan, nan, nan, nan]]

In [169]:
ia.get_movie(imdb_id[7]).keys()

['original title',
 'cast',
 'genres',
 'runtimes',
 'countries',
 'country codes',
 'language codes',
 'color info',
 'aspect ratio',
 'sound mix',
 'certificates',
 'original air date',
 'rating',
 'votes',
 'cover url',
 'plot outline',
 'languages',
 'title',
 'year',
 'kind',
 'directors',
 'writers',
 'producers',
 'composers',
 'cinematographers',
 'editors',
 'editorial department',
 'casting directors',
 'production designers',
 'art directors',
 'set decorators',
 'costume designers',
 'make up department',
 'production managers',
 'assistant directors',
 'art department',
 'sound department',
 'special effects',
 'stunts',
 'camera department',
 'casting department',
 'costume departmen',
 'location management',
 'music department',
 'script department',
 'transportation department',
 'miscellaneous',
 'akas',
 'writer',
 'director',
 'production companies',
 'distributors',
 'other companies',
 'plot',
 'synopsis',
 'canonical title',
 'long imdb title',
 'long imdb canonic

In [171]:
# try pool (pool is way faster - checked)
# fix if condition - but works on vscode

import time

start_time = time.time()
data=pool.map(get_info, range(0,6))
end_time = time.time()

shape=(len(data),len(data[0]))
print(shape)
print(end_time-start_time)

(6, 8)
12.375048875808716


In [172]:
# combine all df into one df
df=pd.DataFrame.from_records(data)
df

0            1           2           3    4    5         6  \
0  Toy Story  373554033.0  30000000.0  12491789.0  8.3   81  841347.0   
1    Jumanji  262797249.0  50000000.0  11084370.0  7.0  104  289088.0   
2        NaN          NaN         NaN         NaN  NaN  NaN       NaN   
3        NaN          NaN         NaN         NaN  NaN  NaN       NaN   
4        NaN          NaN         NaN         NaN  NaN  NaN       NaN   
5       Heat  120000000.0  60000000.0   8445656.0  8.2  170  546627.0   

          7  
0  1.78 : 1  
1  1.85 : 1  
2       NaN  
3       NaN  
4       NaN  
5  2.35 : 1

In [173]:
# format df into my liking
df.columns=inputs
# df.index.name='imdb_id'

In [176]:
# export df into csv
df=df.dropna()
df

title  Cumulative Worldwide Gross      Budget  \
0  Toy Story                 373554033.0  30000000.0   
1    Jumanji                 262797249.0  50000000.0   
5       Heat                 120000000.0  60000000.0   

   Opening Weekend United States  rating runtimes     votes aspect ratio  
0                     12491789.0     8.3       81  841347.0     1.78 : 1  
1                     11084370.0     7.0      104  289088.0     1.85 : 1  
5                      8445656.0     8.2      170  546627.0     2.35 : 1